#### Manipulacao e Transformacao de Tensores

No PyTorch, `torch.reshape()` e `torch.view()` são duas funções usadas para modificar a forma de tensores, mas elas têm diferenças sutis no seu comportamento e uso:

1. **`torch.reshape(tensor, shape(tuple))`**

- **Flexibilidade**: `torch.reshape()` pode ser usado para alterar a forma de um tensor para qualquer outra forma, **desde que o número total de elementos seja o mesmo**. É mais flexível porque, internamente, pode copiar dados para um novo tensor se necessário.
- **Cópia vs. Vista**: Dependendo da disposição original dos dados na memória, `torch.reshape()` pode retornar uma vista (um novo tensor que compartilha dados com o tensor original) ou uma cópia do tensor original. Isso significa que, se você modificar os dados em um tensor retornado por `reshape()`, essas modificações podem ou não ser refletidas no tensor original, dependendo de se uma vista foi retornada ou não.

2. **`tensor.view(tensor, shape)`**

- **Exigência de Contiguidade**: `tensor.view()` exige que o tensor original seja contíguo na memória (ou seja, os elementos são armazenados em sequência sem lacunas) para poder retornar uma vista com a nova forma desejada. Se o tensor original não for contíguo, você receberá um erro ao tentar usar `view()`. Nesse caso, você pode usar `tensor.contiguous()` antes de chamar `view()`.
- **Sempre Retorna uma Vista**: Diferente de `reshape()`, `view()` sempre retorna uma view do tensor original, não uma cópia. Isso significa que qualquer modificação nos dados do tensor retornado por `view()` será refletida no tensor original, e vice-versa.

Quando utilizar `torch.reshape()` ou `torch.view()`

A escolha entre `torch.reshape()` e `torch.view()` geralmente depende de dois fatores: se você precisa garantir que o tensor retornado seja uma view do tensor original e não uma cópia (caso em que você usaria `view()`), e se o tensor original é contíguo na memória (caso em que você pode usar `view()` sem problemas). Se não tiver certeza sobre a contiguidade do tensor ou se precisar de flexibilidade para lidar com tensores não contíguos, `torch.reshape()` é a escolha mais segura. No fim, de maneira geral o mais recomendado a se utilizar é o `torch.reshape()` por ser o mais utilizado e generalista.

In [1]:
import torch

In [8]:
v = torch.ones(50)

v

tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])

In [4]:
v.shape

torch.Size([50])

Vamos, entao agora ver na prática como utilizar o `reshape()` e `view()` e também entender melhor o que ocorre no back-end de ambos estes métodos que tem algumas diferencas entre si.

In [10]:
r_v = torch.reshape(v, (25,2))

In [11]:
r_v.shape

torch.Size([25, 2])

In [12]:
r_v

tensor([[1., 1.],
        [1., 1.],
        [1., 1.],
        [1., 1.],
        [1., 1.],
        [1., 1.],
        [1., 1.],
        [1., 1.],
        [1., 1.],
        [1., 1.],
        [1., 1.],
        [1., 1.],
        [1., 1.],
        [1., 1.],
        [1., 1.],
        [1., 1.],
        [1., 1.],
        [1., 1.],
        [1., 1.],
        [1., 1.],
        [1., 1.],
        [1., 1.],
        [1., 1.],
        [1., 1.],
        [1., 1.]])

Como citado anteriormente podemos modificar o tensor de quaisquer forma desde que seja respeitado a quantidade de elementos. Neste caso temos que o tensor original possui 50 elementos, logo os tensores modificados pelo `reshape` podem possuir quaisquer dimensao desde que a multiplicacao das dimensoes seja 50. Vamos ver na prática:

In [13]:
t_2 = torch.reshape(v,(2,5,5))

In [14]:
t_2.shape

torch.Size([2, 5, 5])

In [15]:
t_2

tensor([[[1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1.]],

        [[1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1.]]])

Entao, veja que o `reshape` é muito flexível e nao foi necessário, se preocupar se a estrutura de dados neste caso o `Tensor`, possuia uma estrutura de dados contígua em memória.

Vamos, agora visualizar na prática como funciona o `view` e vamos comentar um pouco das diferencas sutis na prática entre o uso do `reshape()` e `view()`

In [16]:
r = v.view(2,25)

In [17]:
r.shape

torch.Size([2, 25])

In [18]:
t = v.view(5,5,2)

In [19]:
t.shape

torch.Size([5, 5, 2])

Se observarmos até aqui, se voce perceber nao existe diferenca significativa entre o `reshape` e `view` até aqui uma das diferencas notaveis é que o `view` é na implementacao muito provavelmente um método de instância, pois o view é um atributo do objeto tensor, já o `reashape` é um método do `Pytorch`. 

Além disso, para utilizarmos o `reshape` é necessário passar o tensor o qual queremos modificar e uma tupla contendo o formato ou as dimensoes para qual queremos que este tensor se torne.

Porém como ja mencionado, esta nao é uma das diferencas de fato. A diferenca que existe nao é transparente para o usuário, e ocorre no back-end mais especificamente falando a nível de memória.

Sendo assim, vamos tentar tornar isso transparente ou mais claro, mostrando mais claramente estas diferencas.

Para fazer isso de maneira prática vamos comecar tentando mostrar como funciona a estrutura de dados do tensor.

In [21]:
m = torch.rand(2,3)

m

tensor([[0.1155, 0.6781, 0.7776],
        [0.9788, 0.0502, 0.2136]])

Entao veja que aqui, vamos utilizar o método `is_contiguous()` que irá nos retornar `True` caso a estrutura de dados tenha tensor um tensor onde seus elementos estao organizados de maneira contígua na memória. 

In [22]:
m.is_contiguous()

True

Agora veja que se aplicarmos alguma operacao aritimética ou alguma transformacao neste mesmo tensor, que seus elementos estao organizados de maneira contígua na memória, iremos ver que ele nao será mais contíguo pois a nível de memória estes dados sao deslocados para uma outra regiao memória que "suporte" armazenar e manipular melhor estes dados.

In [23]:
t = m.T

Entao, perceba que aplicamos uma operacao de transposicao, no tensor `m` e vimos que o método `is_contiguous()` retornou False, mostrando que a nível de memória nao temos mais dados contiguos.

In [24]:
t.is_contiguous()

False

Agora, se tentarmos modificar o tensor, utilizando o `view` iremos receber um erro justamente pelo fato de que o tensor nao é contíguo em memória. Veja

In [26]:
t.view(1,2,3)

RuntimeError: view size is not compatible with input tensor's size and stride (at least one dimension spans across two contiguous subspaces). Use .reshape(...) instead.

Entretanto, se tentarmos realizar o mesmo procedimento de transformacao de tensor utilizando o `reshape()` irá funcionar normalmente, pois como mencionado anteriormente, no `reshape()` nao é necessário, se preocupar se a estrutura de dados é contígua em memória.

In [29]:
f = torch.reshape(t,(1,2,3))

f

tensor([[[0.1155, 0.9788, 0.6781],
         [0.0502, 0.7776, 0.2136]]])

In [31]:
type(f)

torch.Tensor

Perceba, agora que após utilizarmos o `reshape()` e armazenarmos esta transformacao, no objeto `f.Tensor` temos que a nível de memória este tensor, agora é contíguo em memória e portanto, se tentarmos realizar uma transformacao utilizando o `view`, agora nao receberemos mais nenhum erro. Por fim, nao podemos afirmar, porém o reshape nos retorna um tensor com dados ou elementos contíguos em memória.

In [30]:
f.is_contiguous()

True

In [32]:
f.view(2,3)

tensor([[0.1155, 0.9788, 0.6781],
        [0.0502, 0.7776, 0.2136]])

Entretanto, mesmo que agora temos um tensor que possui elementos contíguos em memória, se aplicarmos alguma transformacao, novamente teremos um tensor que nao é contíguo em memória.

In [35]:
g = f.T

/var/folders/h0/jqw3wsl561j81vjn6nzz1j740000gn/T/ipykernel_94977/3401940231.py:1: UserWarning: The use of `x.T` on tensors of dimension other than 2 to reverse their shape is deprecated and it will throw an error in a future release. Consider `x.mT` to transpose batches of matrices or `x.permute(*torch.arange(x.ndim - 1, -1, -1))` to reverse the dimensions of a tensor. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/aten/src/ATen/native/TensorShape.cpp:3641.)
  g = f.T


In [36]:
g.is_contiguous()

False

Porém caso, por algum motivo o usuário queira modificar a estrutura de dados do tensor de tal forma que ela se transforme em uma estrutura contígua podemos utilizar o método `contiguous()` E assim, podemos após converter para um tensor contíguos podemos usar livremente o `view`

In [39]:
g_c = g.contiguous()

g_c

tensor([[[0.1155],
         [0.0502]],

        [[0.9788],
         [0.7776]],

        [[0.6781],
         [0.2136]]])

In [40]:
g_c.view(2,3,1)

tensor([[[0.1155],
         [0.0502],
         [0.9788]],

        [[0.7776],
         [0.6781],
         [0.2136]]])

Por fim, vimos que podemos transformar tensores, de diversas maneiras, nao só na estrutura do tensor mas também modificar a estrutura de dados a nível de memória. Entretanto, a recomendacao é usar sempre o `reshape()` como método, padrao para se transformar tensores e utilizar o `view` em situacoes específicas.

##### torch.tril()
Vamos, ver agora o método `torch.tril()`. O que este método faz ? bom este método e responsável por criar um matriz triangular com o mesmo comportamento das matrizes que estudamos em algebra linear.

Legal, mas onde iremos utilizar este tipo de matriz triangular?

Bom, as matrizes triangular, ou melhor dizendo os tensores triangulares sao extremamente úteis quando queremos criar "máscaras" em nossos tensores. Um exemplo de aplicacao extremamente famosa é na implementacao do mecanismo de atencao da arquitetura Transformer do paper "Attetion is all you need"

Vamos entao observar como utilizar o `torch.tril()` na prática

In [2]:
t1 = torch.ones(10,10)

t1

tensor([[1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]])

Veja que ele realizou uma "mascara", o local onde antes eram valores 1 ele trocou por 0 realizando um triangulo na parte superior do tensor. Já na parte inferior do tensor está da mesma maneira

In [3]:
torch.tril(t1)

tensor([[1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [1., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
        [1., 1., 1., 0., 0., 0., 0., 0., 0., 0.],
        [1., 1., 1., 1., 0., 0., 0., 0., 0., 0.],
        [1., 1., 1., 1., 1., 0., 0., 0., 0., 0.],
        [1., 1., 1., 1., 1., 1., 0., 0., 0., 0.],
        [1., 1., 1., 1., 1., 1., 1., 0., 0., 0.],
        [1., 1., 1., 1., 1., 1., 1., 1., 0., 0.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 0.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]])

Mas, um ponto interessante a se salientar em relacao ao `torch.tril()` é que o mesmo possui alguns parametros especiais, como por exemplo o parâmetro `diagonal`. Esse parametro por padrao é 0. Quando o valor de diagonal é zero teremos o comportamento padrao que vimos anteriormente. Caso esse valor seja aumentado, ele irá diminuir o tamanho do triangulo na parte superior do tensor, ou seja, diminuiiremos a quantidade de diagonais que tem o número 0 presente 

In [4]:
torch.tril(t1, diagonal = 0)

tensor([[1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [1., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
        [1., 1., 1., 0., 0., 0., 0., 0., 0., 0.],
        [1., 1., 1., 1., 0., 0., 0., 0., 0., 0.],
        [1., 1., 1., 1., 1., 0., 0., 0., 0., 0.],
        [1., 1., 1., 1., 1., 1., 0., 0., 0., 0.],
        [1., 1., 1., 1., 1., 1., 1., 0., 0., 0.],
        [1., 1., 1., 1., 1., 1., 1., 1., 0., 0.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 0.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]])

Veja que agora como utilizamos **diagonal=2**, o tensor diminui a diagonal de "0" significativamente.

In [5]:
torch.tril(t1,diagonal = 2)

tensor([[1., 1., 1., 0., 0., 0., 0., 0., 0., 0.],
        [1., 1., 1., 1., 0., 0., 0., 0., 0., 0.],
        [1., 1., 1., 1., 1., 0., 0., 0., 0., 0.],
        [1., 1., 1., 1., 1., 1., 0., 0., 0., 0.],
        [1., 1., 1., 1., 1., 1., 1., 0., 0., 0.],
        [1., 1., 1., 1., 1., 1., 1., 1., 0., 0.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 0.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]])

E veja que se utilizarmos valores negativos, o método `torch.tril()`inverte a ordem de crescimento do triangulo, entao a direcao do crescimento da "mascara" agora é do canto superior em direcao ao canto inferior.

In [6]:
torch.tril(t1, diagonal = -7)

tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [1., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
        [1., 1., 1., 0., 0., 0., 0., 0., 0., 0.]])

##### torch.cat() e torch.stack()
Vamos, agora entender melhor sobre dois métodos que sao muito parecidos entre si mas com pequenas sutis diferencas. 

Os método que estamos falando sao o `torch.cat()` e o `torch.stack()`. O `torch.cat()` tem a funcao de concatenar 2 tensores o transformando ou redimensionando em um tensor bem maior, o usuário pode informar a dimensao a qual queremos que os tensores sejam concatenados, porém caso nao seja informada nenhuma dimensao, entao será concatenado através da dimensao 0 que é a padrao. Vamos ver na prática como tudo isso funciona:

In [7]:
t_1 = torch.ones(2,3)
t_2 = torch.zeros(2,3)

In [8]:
t_1

tensor([[1., 1., 1.],
        [1., 1., 1.]])

In [9]:
t_2

tensor([[0., 0., 0.],
        [0., 0., 0.]])

Entao criamos, dois tensores e agora vamos concatenar ambos os tensores utilizando primeiramente o `torch.cat()`. Para usar, o método devemos passar como argumento uma tupla contendo ambos os tensores que queremos concatenar. 

In [10]:
r = torch.cat((t_1, t_2))

r

tensor([[1., 1., 1.],
        [1., 1., 1.],
        [0., 0., 0.],
        [0., 0., 0.]])

In [11]:
r.shape

torch.Size([4, 3])

Como foi possível ver, na célula de código acima nao informamos nada, além da tupla contendo ambos os tensores que queremos concatenar. Porém, podemos informar também a dimensao para qual queremos que essa concatenacao ocorra, caso nao seja informado nenhuma `dim` a dimensao será igual a `dim = 0` que neste caso irá realizar concatenacao na dimensao das linhas. Vamos ver um exemplo informando o parâmetro `dim` de maneira explicita.

Veja que abaixo o exemplo que temos utilizaremos o `dim = 1` e ao utilizarmos este valor de `dim = 1` a concatencao será feita em relacao as colunas ou seja o que está de fato ocorrendo aqui é que vamos ter o tensor `t_1` e iremos "grudar" "unir" este `t_1` a partir da sua última coluna ao tensor `t_2`.

In [12]:
r1 = torch.cat((t_1, t_2), dim = 1)

r1

tensor([[1., 1., 1., 0., 0., 0.],
        [1., 1., 1., 0., 0., 0.]])

Veja que aqui de fato, ocorre o que mencionamos anteriormente e uma das mais de "provar" isto é utilizar o método de instancia `shape` que irá mostrar o formato do Tensor. E aqui o que vemos é isso pois temos um Tensor com formato `[2,6]` ou seja concatenei 2 Tensores `[2,3]` e como passamos a `dim = 1`. Logo, este Tensor resultante, terá a dimensao como mostramos pois será concatenado a partir da dimensao das colunas, logo de fato é algo obvio de perceber pois teremos **"3 colunas a serem concatenadas".**

In [13]:
r1.shape

torch.Size([2, 6])

Veja que caso seja informado um valor de `dim` que seja maior do que o número de dimensoes existentes entre ambos os tensores teremos um erro.

In [14]:
r2_test = torch.cat((t_1, t_2), dim = 2)

IndexError: Dimension out of range (expected to be in range of [-2, 1], but got 2)

Vamos, agora abordar o `torch.stack()` e ver de fato as diferencas existentes entre eles.

Bom, o **torch.stack()** empilha os tensores ao longo de uma nova dimensão, diferentemente do `torch.cat()` que concatena os Tensores. Entao um tem a ideia de concatenar e o `torch.stack` tem a ideia de empilhar, logo isso provoca um aumento no otal de dimensões do tensor resultante.

In [15]:
m_1 = torch.ones(2,3)
m_2 = torch.zeros(2,3)

In [16]:
m_1

tensor([[1., 1., 1.],
        [1., 1., 1.]])

In [17]:
m_2

tensor([[0., 0., 0.],
        [0., 0., 0.]])

Agora, com os tensores criados vamos, aplicar o `torch.stack()` e ver como tudo isso funciona. O **stack** funciona de maneira extremamente parecida com o `torch.cat()` ou seja vamos informar um **tupla** contendo os Tensores que iremos empilhar. Vamos ver entao na prática como ele funciona.

In [19]:
r_tensor = torch.stack((m_1, m_2))

r_tensor

tensor([[[1., 1., 1.],
         [1., 1., 1.]],

        [[0., 0., 0.],
         [0., 0., 0.]]])

In [20]:
r_tensor.shape

torch.Size([2, 2, 3])

In [21]:
r2_tensor = torch.stack((m_1, m_2), dim = 1)

r2_tensor

tensor([[[1., 1., 1.],
         [0., 0., 0.]],

        [[1., 1., 1.],
         [0., 0., 0.]]])

In [22]:
r2_tensor.shape

torch.Size([2, 2, 3])

Agora, vamos falar de outro metodo de transformacao de `Tensores` que neste caso se trata do método `torch.chunk`.

O `torch.chunk` tem a funcao, de dividir um tensor em um número definido de partes, ou seja, consegue quebrar o tensor em varios "pedacos" de tensores menores da maneira como se queira. Entretanto, pode ocorrer que ele seja quebrado em partes de tamanho irregular caso o tensor nao puder ser dividido igualmente.

Vamos ver um pouco melhor como isso funciona.

In [2]:
import torch
v = torch.zeros(9)

v

tensor([0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [5]:
ch = torch.chunk(v, chunks=3)

ch

(tensor([0., 0., 0.]), tensor([0., 0., 0.]), tensor([0., 0., 0.]))

Perceba, que criamos nas células anteriores um tensor de ordem 1, contendo 9 elementos, e quando, utilizamos `chunks = 3`, foi possivel quebrar o tensor em exatas 3 partes iguais.

Entretanto, se observarmos a célula abaixo, será possível ver que quando utilizamos `chunks = 5`, foi possível obter 4 tensores contendo 2 elementos cada um e 1 tensor contendo apenas 1 elemento, e de fato isso ocorreu, pois criamos um tensor de ordem 1 de 9 elementos. E esta é a caracteristica do `torch.chunk()` ele quebra o tensor em uma quantidade informada como parametro da funcao. Uma observacao importante é que podemos, realizar o chunk em torno de uma dimensao informando o parametro `dim`. Neste caso seria redundante pois estamos trabalhando com tensores de ordem 1 e portanto só possuimos esta dimensao para realizar o chunck, entretanto a seguir iremos trabalhar com tensores de ordem 3 ou superior e será possível ver como realizar o chunk através de uma dimensao.

In [7]:
ch = torch.chunk(v, chunks = 5)

ch

(tensor([0., 0.]),
 tensor([0., 0.]),
 tensor([0., 0.]),
 tensor([0., 0.]),
 tensor([0.]))

In [11]:
tt = torch.ones(2,3,10)
tt.shape

torch.Size([2, 3, 10])

In [12]:
a,b = torch.chunk(tt, chunks = 2, dim = 2)

In [13]:
a.shape

torch.Size([2, 3, 5])

In [14]:
b.shape

torch.Size([2, 3, 5])

In [15]:
a,b,c = torch.chunk(tt, chunks = 3, dim = 2)

a.shape, b.shape, c.shape

(torch.Size([2, 3, 4]), torch.Size([2, 3, 4]), torch.Size([2, 3, 2]))

Entao, veja que ao usar, o parametro `dim`, e possível quebrar o tensor a partir daquela dada dimensao que foi passada como parametro, e assim como já vimos o método `torch.chunk()` ele irá quebrar em partes iguais, até que seja possível. E veja que isso ocorre pois se observarmos o formato dos tensores `a` e `b`, ambos possuem dimensao do tipo [2,3,4] enquanto que o tensor `c`possui uma dimensao [2,3,2] e isso ocorre pois fizemos, um chunk = 3, na `dim = 2` em um tensor `tt` de 10 elementos na dimensao 2. E portanto, isso nos retorna um "comportamento" que ja era esperado como retorno do método. 